In [12]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib.parse import urljoin
import time

In [13]:
url = requests.get('http://www.ufcstats.com/statistics/events/completed?page=all')
soup = BeautifulSoup(url.text, 'html.parser')
eventLinks = []
for link in soup.select(".b-statistics__table-row a"):
    eventUrl = link["href"]
    eventLinks.append(eventUrl)

for item in eventLinks:
    print(item)

http://www.ufcstats.com/event-details/ce7871949b0ed2bf
http://www.ufcstats.com/event-details/e6015889f50075d2
http://www.ufcstats.com/event-details/80dbeb1dd5b53e64
http://www.ufcstats.com/event-details/39f68882def7a507
http://www.ufcstats.com/event-details/81ddc98fceb30086
http://www.ufcstats.com/event-details/72c9c2eadfc3277e
http://www.ufcstats.com/event-details/ad23903ef3af7406
http://www.ufcstats.com/event-details/e955046551f8c7dd
http://www.ufcstats.com/event-details/daff32bc96d1eabf
http://www.ufcstats.com/event-details/585f9ffdb0cd0466
http://www.ufcstats.com/event-details/3ab0df1ddd5f1dc2
http://www.ufcstats.com/event-details/221b2a3070c7ce3e
http://www.ufcstats.com/event-details/6a0047ac8c99fece
http://www.ufcstats.com/event-details/5bdcbd8dd681a257
http://www.ufcstats.com/event-details/eee8efec7b951d84
http://www.ufcstats.com/event-details/079488710f012779
http://www.ufcstats.com/event-details/0c01568b1ac77bff
http://www.ufcstats.com/event-details/ee0b69e307c857e5
http://www

In [ ]:
fightData = []

for eventUrl in eventLinks[0:600]:
    eventURLNext = requests.get(eventUrl)
    eventSoup = BeautifulSoup(eventURLNext.text, "html.parser")

    # get event name & date
    event_name = eventSoup.select_one(".b-content__title").text.strip()
    event_date = eventSoup.select_one(".b-list__box-list-item:first-child").text.strip()

    # find the fight table
    fightTable = eventSoup.find("table", class_="b-fight-details__table")
    
    if fightTable:
        # loop through all the rows and extract the relevant data
        for row in fightTable.find_all("tr")[1:]:  # Skip header row
            cols = [col.text.strip() for col in row.find_all("td")]
            if len(cols) > 6:
                winnerTagFake = cols[0].split("\n")
                winnerTag = winnerTagFake[0].strip()

                fighters = cols[1].split("\n")
                
                winner = fighters[0].strip()  
                loser = fighters[-1].strip()  
                weightClass = cols[6]
                method = cols[7].split("\n")
                trueMethod = method[0].strip()
                roundEnd = cols[8]
                time = cols[9]
                titleFight = "No"
                beltImage = row.find("img", src=lambda x: x and "belt.png" in x)
                
                if beltImage:
                    titleFight = "Yes"  # If the belt image is found, it's a title fight

               
                # Append fight data to the list
                fightData.append([winnerTag, winner, loser, weightClass, titleFight, trueMethod, roundEnd, time])


In [5]:
initialElo = 1500
kFactor = 48 
eloRatings = {}

In [6]:
def expectedScore(ratingA, ratingB):
    return 1 / (1 + 10 ** ((ratingB - ratingA) / 400))

In [7]:
def updateElo(winner, loser, eloRatings, kFactor, isTitleFight):
    if winner not in eloRatings:
        eloRatings[winner] = initialElo
    if loser not in eloRatings:
        eloRatings[loser] = initialElo

    ratingWinner = eloRatings[winner]
    ratingLoser = eloRatings[loser]

    # calculate expected scores
    expectedWinner = expectedScore(ratingWinner, ratingLoser)
    expectedLoser = expectedScore(ratingLoser, ratingWinner)

    # Update Elo ratings
    if isTitleFight == "Yes":   
        eloRatings[winner] = ratingWinner + (kFactor * (1 - expectedWinner) * 2)
        eloRatings[loser] = ratingLoser + kFactor * (0 - expectedLoser)
    else:
        eloRatings[winner] = ratingWinner + kFactor * (1 - expectedWinner)
        eloRatings[loser] = ratingLoser + kFactor * (0 - expectedLoser)

In [8]:
def processFights(fight_data):
    for fight in reversed(fightData):
        winner = fight[1]
        loser = fight[2]
        isTitleFight = fight[4]
        if winner != "Draw":
            updateElo(winner, loser, eloRatings, kFactor, isTitleFight)

In [9]:
processFights(fightData)
sortedElo = sorted(eloRatings.items(), key=lambda x: x[1], reverse=True)
for fighter, rating in sortedElo:
    print(f"{fighter}: {rating:.2f}")

NameError: name 'fightData' is not defined

In [ ]:
for item in fightData:
    print(item)

['win', 'Jared Cannonier', 'Gregory Rodrigues', 'Middleweight', 'No', 'KO/TKO', '4', '0:21']
['win', 'Youssef Zalal', 'Calvin Kattar', 'Featherweight', 'No', 'U-DEC', '3', '5:00']
['win', 'Edmen Shahbazyan', 'Dylan Budka', 'Middleweight', 'No', 'KO/TKO', '1', '1:35']
['win', 'Nazim Sadykhov', 'Ismael Bonfim', 'Lightweight', 'No', 'KO/TKO', '1', '5:00']
['win', 'Andre Petroski', 'Rodolfo Vieira', 'Middleweight', 'No', 'U-DEC', '3', '5:00']
['win', 'Jose Delgado', 'Connor Matthews', 'Featherweight', 'No', 'KO/TKO', '1', '2:58']
['win', 'Angela Hill', 'Ketlen Souza', "Women's Strawweight", 'No', 'S-DEC', '3', '5:00']
['win', 'Rafael Estevam', 'Jesus Aguilar', 'Flyweight', 'No', 'U-DEC', '3', '5:00']
['win', 'Gabriel Bonfim', 'Khaos Williams', 'Welterweight', 'No', 'SUB', '2', '4:58']
['win', 'Elijah Smith', 'Vince Morales', 'Bantamweight', 'No', 'U-DEC', '3', '5:00']
['win', 'Valter Walker', "Don'Tale Mayes", 'Heavyweight', 'No', 'SUB', '1', '1:17']
['win', 'Jacqueline Cavalcanti', 'Julia